# Stage 1 - Scraping Lift Off!

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import os
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\willt\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.liftoff_db
collection = db.items

## NASA Mars News

In [5]:
# Source Scrape URL 
news_url = 'https://redplanetscience.com/'
browser.visit(news_url)
time.sleep(3)

In [6]:
# Retrieve page
news_retrieve = requests.get(news_url)

# Create BeautifulSoup object
news_soup = BeautifulSoup(news_retrieve.text, 'html.parser')

news_html = browser.html
news_results = BeautifulSoup(news_html, 'html.parser')

In [7]:
# Retrieve headline and flavor text
news_title = news_results.find('div', class_='content_title').get_text()
news_teaser = news_results.find('div', class_='article_teaser_body').get_text()

print(news_title)
print(news_teaser)

HiRISE Views NASA's InSight and Curiosity on Mars
New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.


## JPL Mars Space Images - Featured Image

In [8]:
# Source Scrape URL 
images_url = 'https://spaceimages-mars.com/'
browser.visit(images_url)
time.sleep(3)

In [9]:
# Retrieve page
images_retrieve = requests.get(images_url)

# Create BeautifulSoup object
images_soup = BeautifulSoup(images_retrieve.text, 'html.parser')

images_html = browser.html
images_results = BeautifulSoup(images_html, 'html.parser')

In [10]:
# Retrieve Current Header Image from URL 
featured_image = images_results.find('img', class_='headerimage')
featured_image = featured_image['src']
featured_image

'image/featured/mars2.jpg'

In [11]:
#Combine URl and header image
featured_image_url = images_url + featured_image
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


## Mars Facts

In [12]:
# Source Scrape URL 
facts_url = 'https://galaxyfacts-mars.com/'

In [13]:
#Set table data into Dataframe
tables = pd.read_html(facts_url)
facts_df = tables[0]
facts_df.columns = ["Description","Mars_Data","Earth_Data"]
facts_df.set_index("Description", inplace=True)
facts_df

,Mars_Data,Earth_Data
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
#Converted DataFrame into HTML string
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars_Data</th>
      <th>Earth_Data</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [15]:
clean_html = facts_html.replace('\n', '')
clean_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars_Data</th>      <th>Earth_Data</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Mars - Earth Comparison</th>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [16]:
# Save file as html 
mars_html = facts_df.to_html('mars_table.html')
mars_html

## Mars Hemishperes 

In [17]:
hemi_dict = []

### Valles Marineris 

In [18]:
# Source Scrape URL 
hemi_url = 'https://marshemispheres.com/'
browser.visit(hemi_url)
time.sleep(3)

In [19]:
# Retrieve page
browser.links.find_by_partial_text('Valles Marineris').click()

hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [20]:
# Get URL of page to be scraped
img_link = hemi_soup.find('img', class_='wide-image').get('src')

#Pull Hemisphere title 
valles_title = hemi_soup.find('h2', class_='title').text

# Combine url & img link to make full html link=
valles_img = hemi_url + img_link

browser.back()

valles_dict = {'title:' + valles_title, 'img_url:' + valles_img}
hemi_dict.append(valles_dict)

valles_dict

{'img_url:https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
 'title:Valles Marineris Hemisphere Enhanced'}

### Syrtis

In [21]:
# Source Scrape URL 
hemi_url = 'https://marshemispheres.com/'
browser.visit(hemi_url)
time.sleep(3)

In [22]:
# Retrieve page
browser.links.find_by_partial_text('Syrtis').click()

hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [23]:
# Get URL of page to be scraped
img_link = hemi_soup.find('img', class_='wide-image').get('src')

#Pull Hemisphere title 
syrt_title = hemi_soup.find('h2', class_='title').text

# Combine url & img link to make full html link=
syrt_img = hemi_url + img_link

browser.back()

syrt_dict = {'title:' + syrt_title, 'img_url:' + syrt_img}
hemi_dict.append(syrt_dict)

syrt_dict

{'img_url:https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'title:Syrtis Major Hemisphere Enhanced'}

### Schiaparelli

In [24]:
# Source Scrape URL 
hemi_url = 'https://marshemispheres.com/'
browser.visit(hemi_url)
time.sleep(3)

In [25]:
# Retrieve page
browser.links.find_by_partial_text('Schiaparelli').click()

hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [26]:
# Get URL of page to be scraped
img_link = hemi_soup.find('img', class_='wide-image').get('src')

#Pull title 
schia_title = hemi_soup.find('h2', class_='title').text

# Get html link
schia_img = hemi_url + img_link

browser.back()

schia_dict = {'title:' + schia_title, 'img_url:' + schia_img}
hemi_dict.append(schia_dict)

schia_dict

{'img_url:https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'title:Schiaparelli Hemisphere Enhanced'}

### Cerberus 

In [27]:
# Source Scrape URL 
hemi_url = 'https://marshemispheres.com/'
browser.visit(hemi_url)
time.sleep(3)

In [28]:
# Retrieve page
browser.links.find_by_partial_text('Cerberus').click()

hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [29]:
# Get URL of page to be scraped
img_link = hemi_soup.find('img', class_='wide-image').get('src')

#Pull title 
cerb_title = hemi_soup.find('h2', class_='title').text

# Get html link
cerb_img = hemi_url + img_link

browser.back()

cerb_dict = {'title:' + cerb_title, 'img_url:' + cerb_img}
hemi_dict.append(cerb_dict)

cerb_dict

{'img_url:https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'title:Cerberus Hemisphere Enhanced'}

In [30]:
print(hemi_dict)

[{'title:Valles Marineris Hemisphere Enhanced', 'img_url:https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}, {'title:Syrtis Major Hemisphere Enhanced', 'img_url:https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title:Schiaparelli Hemisphere Enhanced', 'img_url:https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title:Cerberus Hemisphere Enhanced', 'img_url:https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}]


In [31]:
browser.quit()